In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
191,This program would be useful for training hard...,negative
426,Well you take O.J. Simpson as a all american s...,positive
796,"""Don't Drink the Water"" is an unbelievably bad...",negative
5,So we saw this on DVD at our apartment here in...,positive
988,With a catchy title like the Butcher of Plainf...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
191,program would useful training hardened felon b...,negative
426,well take o j simpson american soldier turned ...,positive
796,don t drink water unbelievably bad film based ...,negative
5,saw dvd apartment paris exchange program laugh...,positive
988,catchy title like butcher plainfield ed gein v...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    258
positive    242
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
191,program would useful training hardened felon b...,0
426,well take o j simpson american soldier turned ...,1
796,don t drink water unbelievably bad film based ...,0
5,saw dvd apartment paris exchange program laugh...,1
988,catchy title like butcher plainfield ed gein v...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [18]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/faizalsuhel874742/MLOps-Capstone-Project.mlflow')
dagshub.init(repo_owner='faizalsuhel874742', repo_name='MLOps-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-09-08 00:20:40,747 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/faizalsuhel874742/MLOps-Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "faizalsuhel874742/MLOps-Capstone-Project"

2025-09-08 00:20:40,756 - INFO - Initialized MLflow to track repo "faizalsuhel874742/MLOps-Capstone-Project"


Repository faizalsuhel874742/MLOps-Capstone-Project initialized!

2025-09-08 00:20:40,759 - INFO - Repository faizalsuhel874742/MLOps-Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/652b439ac0904fc1bd732a06764e5dbd', creation_time=1757269398471, experiment_id='0', last_update_time=1757269398471, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [22]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "sklearn-model")
        # Save locally
        mlflow.sklearn.save_model(model, "sklearn-model_1")
        # Log as an artifact (this works on DagsHub)
        mlflow.log_artifacts("sklearn-model_1", artifact_path="model")
        

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-08 00:21:32,259 - INFO - Starting MLflow run...
2025-09-08 00:21:32,897 - INFO - Logging preprocessing parameters...
2025-09-08 00:21:34,575 - INFO - Initializing Logistic Regression model...
2025-09-08 00:21:34,577 - INFO - Fitting the model...
2025-09-08 00:21:34,595 - INFO - Model training complete.
2025-09-08 00:21:34,596 - INFO - Logging model parameters...
2025-09-08 00:21:35,175 - INFO - Making predictions...
2025-09-08 00:21:35,175 - INFO - Calculating evaluation metrics...
2025-09-08 00:21:35,186 - INFO - Logging evaluation metrics...
2025-09-08 00:21:40,501 - INFO - Saving and logging the model...
2025-09-08 00:21:48,135 - INFO - Model training and logging completed in 15.24 seconds.
2025-09-08 00:21:48,135 - INFO - Accuracy: 0.68
2025-09-08 00:21:48,136 - INFO - Precision: 0.6428571428571429
2025-09-08 00:21:48,137 - INFO - Recall: 0.75
2025-09-08 00:21:48,139 - INFO - F1 Score: 0.6923076923076923


🏃 View run legendary-auk-51 at: https://dagshub.com/faizalsuhel874742/MLOps-Capstone-Project.mlflow/#/experiments/0/runs/76c17156235248d394a5aaacf2b58aaa
🧪 View experiment at: https://dagshub.com/faizalsuhel874742/MLOps-Capstone-Project.mlflow/#/experiments/0
